In [ ]:
# web crawler for collecting article 

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request as req 
from urllib.parse import quote
import re
import csv
import datetime
import os
import time

# A function of collecting articles containing a specific company's name from 'fromd' to 'tod'
# e.g. getArticles("네이버", "2020.01.01", "2020.01.31)
def getArticles(companies, fromd, tod):
    print("Starttime:",datetime.datetime.now())
    company = quote(companies)         #url encoding
    dates = [fromd,tod]                #from to
    fromdate = dates[0]
    todate = dates[1]
    fromdate = fromdate.split('.')     
    todate = todate.split('.')
    fromdate=datetime.datetime(int(fromdate[0]), int(fromdate[1]), int(fromdate[2]))
    todate=datetime.datetime(int(todate[0]), int(todate[1]), int(todate[2]))
    dt = datetime.timedelta(-1)

    # iteration by day counts
    while fromdate <= todate:
        time.sleep(0.1)
        date = fromdate
        urldate = date.strftime('%Y.%m.%d')
        url2 = "https://search.naver.com/search.naver?&where=news&query="+company+"&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds="+urldate+"&de="+urldate+"&docid=&nso=so:r,a:all&mynews=0&mson=0&refresh_start=0&related=0"
        requrl2 = req.Request(url2, headers = {'User-Agent':'Mozilla/5.0'})
        res2 = req.urlopen(requrl2).read().decode('utf8')             #url decoding
        soup2 = BeautifulSoup(res2,'html.parser')
        
        if not soup2.find('div',{'class':'title_desc'}) == None:
            a = soup2.find('div',{'class':'title_desc'}).text        
            maxContentNum = int(re.compile(r'/(.*)건$').search(a).group(1).split(' ')[-1].replace(',',''))  
            maxPage = int(maxContentNum/10) + 1
#            print(maxContentNum, maxPage)
        
        # List saving article chunk
            press = []
            title = []
            content = []
            link = []

            for i in range(maxPage):
                url = "https://search.naver.com/search.naver?&where=news&query="+company+"&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds="+urldate+"&de="+urldate+"&docid=&nso=so:r,a:all&mynews=0&cluster_rank=1001&start="+str(i*10 +1)+"&refresh_start=0"
                requrl = req.Request(url, headers = {'User-Agent':'Mozilla/5.0'})
                res = req.urlopen(requrl).read().decode('utf8')
                soup = BeautifulSoup(res,'html.parser')
                s=soup.find('ul',{'class':'type01'})
                
                if s != None:
                    for inline in s.findAll('dd',{'class':'txt_inline'}):
                        naver = inline.find('a')
                          
                # Designated one big search engine Naver, which is the biggest in Korea, to collect articles since HTML Tags vary across all newspapers
                        if "네이버뉴스" == naver.text and "전자신문" != inline.find('span',{'class':'_sp_each_source'}).text:
                            siteurl = naver.get('href')
                            siteurl3 = req.Request(siteurl, headers = {'User-Agent':'Mozilla/5.0'})
                            try3 = 0
                            error_count = 0
                            while try3 < 5:
                                try:
                                    res3 = req.urlopen(siteurl3)
                                    try3 = 5
                                except Exception:
                                    try3 = try3 + 1
                                    error_count = error_count + 1
                                    print(try3)
                                    time.sleep(5)
                                    pass
                                
                            if error_count >= 5:
                                raise NotImplementedError
                            try:
                                soup3 = BeautifulSoup(res3,'html.parser')
                            except NotImplementedError:
                                soup3 = BeautifulSoup(res3,'lxml')
                            finally:
                            
                                try:
                                    tit = soup3.find('div',{'class':'article_info'}).find('h3').text
                                    cont = soup3.find('div',{'id':'articleBodyContents'}).text.split('{}')[-1]
                            
                                    if tit.upper().count(companies.upper()) > 0 or cont.upper().count(companies.upper()) > 0:
                                        link.append(siteurl)
                                        press.append(inline.find('span',{'class':'_sp_each_source'}).text.strip())
                                        title.append(tit.strip())
                                        content.append(cont.strip())
                                except AttributeError:
                                    pass

#         data = {"press":press, "link":link, "title":title,"content":content}
#         data = pd.DataFrame(data)
#         data.to_csv(companies+'_'+urldate.replace('.','')+'.csv',encoding=None)
#         return data.head()
            # Save collected data
            path = "D:/projects/articles/"+companies
            if not os.path.exists(path):
                os.mkdir(path)
            if (len(press)==len(title))&(len(content)==len(press))&(len(press)==len(link))&(len(press)*len(content)*len(link)*len(title)!=0):
                with open(path+'/'+companies+'_'+urldate.replace('.','')+'.csv', 'w',newline='') as outputFile:
                    outputWriter = csv.writer(outputFile)
            # Remove characters arising CP949 encoding problems
                    for p in range(len(press)):
                        try:
                            outputWriter.writerow([link[p].replace('\xa0','').replace('\xa9','').replace('\u301c','').replace('\u5d1b','').replace('\u22c5','').replace('\u7f51','').strip(),
                                                   title[p].replace('\xa0','').replace('\xa9','').replace('\u301c','').replace('\u5d1b','').replace('\u22c5','').replace('\u7f51','').strip(),
                                                   press[p].replace('\xa0','').replace('\xa9','').replace('\u301c','').replace('\u5d1b','').replace('\u22c5','').replace('\u7f51','').strip(),
                                                   content[p].replace('\xa0','').replace('\xa9','').replace('\u301c','').replace('\u5d1b','').replace('\u22c5','').replace('\u7f51','').strip()])
                        except UnicodeEncodeError:
                            pass
        
        fromdate = fromdate - dt

        print("Endtime: ",datetime.datetime.now())

In [ ]:
# Embedding words into vectors to get similarit features
# Get articles for a company from directory
import os, csv
from konlpy.tag import Twitter
import konlpy
from gensim.models import word2vec

def getContents(path):
    files = os.listdir(path)
    file1 = [file for file in files]

    contents=[]
    for filename in file1:
        exampleFile = open(path + filename)
        exampleReader = csv.reader(exampleFile)
        for row in exampleReader:
            contents.append(row[3])
    return contents

path = "D:/projects/articles/하이닉스/"
train_data = getContents(path)

# Tokenize textual data
def tokenize(doc):
    return ['/'.join(t) for t in Twitter().pos(doc, norm=True, stem=True)] 
            
train_docs = [row for row in train_data]
sentences = [tokenize(d) for d in train_docs]

# Embed words into vectors
def selectNoun(w):
    ww = []
    for words in w:
        splited = words.split('/')
        if(splited[-1]=='Noun') and (len(splited[0])>=2):
            ww.append(words)
    return ww
sentences = [selectNoun(s) for s in sentences]
model = word2vec.Word2Vec(sentences)
model.init_sims(replace=True)

# Example of result with a word '우려'
model.most_similar(['우려/Noun'])